# Definindo um problema para otimizar

Agora vamos montar um exemplo simples de usar um algoritmo genético em Python. Vamos otimizar um problema muito simples: tentar criar uma lista de Nnúmeros iguais X quando somados.

Se definirmos N = 5e X = 200, em seguida, todos estes seriam soluções adequadas.

Referência - [Will Larson](https://lethain.com/genetic-algorithms-cool-name-damn-simple/) - site: *Irrational Exuberance!*

### Exemplo:

Se definirmos N = 5e X = 200, em seguida, todos estes seriam soluções adequadas.

lst  =  [ 40 , 40 , 40 , 40 , 40 ] 
lst  =  [ 50 , 50 , 50 , 25 , 25 ] 
lst  =  [ 200 , 0 , 0 , 0 , 0 ]

# Solução
Cada solução sugerida para um algoritmo genético é referida como um indivíduo . Em nosso problema atual, cada lista de N números é um indivíduo.

### import libraries

In [14]:
from random import randint, random
from operator import add
import functools 

In [15]:
def individual(length, min, max):
    "create a member of the population"
    return[ randint(min,max) for x in range(length)] 

#print(individual(5,0,100))

A coleta de todos os indivíduos é referida como nossa população

In [16]:
def population(count, length, min, max):    
    """
    Create a number of individuals (i.e. a population).

    count: the number of individuals in the population
    length: the number of values per individual
    min: the min possible value in an individual's list of values
    max: the max possible value in an individual's list of values
    
    """
    return [ individual(length, min, max) for x in range(count)]

#print(population(3,5,0,100))

Em seguida, precisamos de uma maneira de julgar a eficácia de cada solução; julgar a aptidão de cada indivíduo. Para este problema, queremos que a adequação seja uma função da distância entre a soma de um número individual e o número alvo X.

Podemos implementar a função de adequação da seguinte maneira:

In [17]:
def fitness(individual, target):
    """
    Determina a aptidão de um indivíduo. Mais baixo é melhor

    indivíduo: o indivíduo para avaliar
    alvo: a soma dos números que os indivíduos estão buscando
    """
    sum = functools.reduce(add, individual, 0) #adicionei a biblioteca functools (não tinha no original)
    return abs(target-sum)

#x = individual(5,0,100)
#print(fitness(x, 200))

Também é útil criar uma função que determine a aptidão média de uma população .

In [18]:
def grade(pop, target):	
    "Encontre aptidão média para uma população"
    summed = functools.reduce(add, (fitness(x, target) for x in pop), 0)
    return summed / (len(pop) * 1.0)

#x = population(3,5,0,100)
#target = 200
#print(grade(x, target))

Agora precisamos de uma maneira de evoluir nossa população de uma geração para a próxima.

### Evolução

Aqui é o secreto dos algoritmos genéticos. Considere uma população de alces que são impiedosamente caçados por um bando de lobos. A cada geração, os mais fracos são devorados pelos lobos e, os alces mais fortes se reproduzem e geram filhos. Abstraia um pouco essas ideias e podemos implementar o mecanismo de evolução.

1. Para cada geração, teremos uma parte dos indivíduos com melhor desempenho, conforme julgado pela nossa função de fitness. Esses indivíduos de alto desempenho serão os pais da próxima geração.

Também vamos selecionar aleatoriamente alguns indivíduos com menores desempenho, isso é importante para promover a diversidade genética. Abandonando a metáfora, um dos perigos dos algoritmos de otimização é ficar preso em um máximo local e, consequentemente, ser incapaz de encontrar o máximo real. Ao incluir alguns indivíduos dessa forma, diminuímos nossa probabilidade de ficarmos presos.

2. Reúna os pais para repovoar a população até o tamanho desejado (se você escolher os 20 principais indivíduos em uma população de 100 pessoas, será necessário criar 80 novos filhos por meio da reprodução). No nosso caso, a criação é bem básica: pegue os primeiros N/2 dígitos do pai e os últimos N/2 dígitos da mãe.

Não há problema em ter uma raça mãe várias vezes, mas um pai nunca deve ser pai e mãe de uma criança.

3. Junte os pais e filhos para constituir a população da próxima geração.

4. Finalmente, nós mutamos uma pequena porção aleatória da população. O que isto significa é ter uma probabilidade de modificar aleatoriamente cada indivíduo.

Juntando tudo, o código para evoluir uma geração pode ser implementado assim:

In [19]:
def evolve(pop, target, retain=0.2, random_select=0.5, mutate=0.01):
    graded = [(fitness(x, target), x) for x in pop]
    graded = [x[1] for x in sorted(graded)]
    retain_length = int(len(graded) * retain)
    parents = graded[:retain_length]

    # Adicionar aleatoriamente outros indivíduos para promover diversidade genética
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)

    # Mutate some indivíduos
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            # esta mutação não é ideal, porque
            # restringe o intervalo de valores possíveis,
            # mas a função não tem conhecimento do min / max
            # valores usados para criar os indivíduos
            individual[pos_to_mutate] = randint(min(individual), max(individual))

    # crossover parents para criar filhos
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length - 1)
        female = randint(0, parents_length - 1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = len(male) / 2
            #child = male[:half] + female[half:] #original
            child = male[:int(half)] + female[int(half):]
            children.append(child)

    parents.extend(children)
    return parents

### Testando-o

Aqui está uma maneira simples de usar o código que escrevemos:

In [20]:
target = 371
p_count = 100
i_length = 5
i_min = 0
i_max = 100
p = population(p_count, i_length, i_min, i_max)


fitness_history = [grade(p, target),] 


for i in range(100):
    p = evolve(p, target)
    fitness_history.append(grade(p, target))

for datum in fitness_history:
    print (datum)


128.39
123.03
108.85
87.2
75.45
53.88
50.21
43.24
39.6
34.41
34.96
27.73
30.43
24.59
30.06
31.73
29.68
28.73
26.74
24.83
23.37
22.65
18.98
17.11
12.21
11.59
9.03
7.22
6.44
3.81
3.02
2.32
0.94
1.27
1.34
1.26
1.11
1.8
1.2
0.94
0.49
0.45
0.34
0.64
0.23
0.39
0.24
0.08
0.24
0.62
0.55
0.16
0.16
0.67
1.09
1.4
1.57
1.79
0.99
1.53
2.05
1.84
2.26
1.84
0.99
0.87
0.65
0.22
0.19
0.3
0.22
1.11
1.23
1.6
1.54
1.4
0.0
0.0
0.0
0.04
0.05
0.15
0.24
0.12
0.36
0.62
0.74
0.19
0.89
0.0
0.0
0.56
0.41
0.13
0.45
0.45
1.02
0.57
0.73
1.41
2.41


Executando esse código, você poderá ver a adaptação das gerações gradualmente (mas não deterministicamente) aproximar-se de zero.

Com 20% de sobrevivência (mais 5% de outros indivíduos) e 1% de mutação, levou poucas gerações para chegar a uma solução perfeita. Então o algoritmo corre alegremente em círculos enquanto você permitir que as mutações continuem. Mas esta é uma sensação boa, certo? Se levamos apenas meia hora para resolver um problema dessa magnitude, imagine o que poderíamos fazer com um dia. 